Pentru a rula următoarele notebook-uri, dacă nu ați făcut-o deja, trebuie să implementați un model care folosește `text-embedding-ada-002` ca model de bază și să setați numele implementării în fișierul .env ca `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

În continuare, vom încărca Indexul de Încapsulare într-un Dataframe Pandas. Indexul de Încapsulare este stocat într-un fișier JSON numit `embedding_index_3m.json`. Indexul de Încapsulare conține Încapsulările pentru fiecare dintre transcrierile YouTube până la sfârșitul lunii octombrie 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

În continuare, vom crea o funcție numită `get_videos` care va căuta în Indexul de Embedding pentru interogare. Funcția va returna primele 5 videoclipuri care sunt cele mai similare cu interogarea. Funcția funcționează astfel:

1. Mai întâi, se creează o copie a Indexului de Embedding.
2. Apoi, se calculează Embedding-ul pentru interogare folosind API-ul OpenAI Embedding.
3. Ulterior, se creează o coloană nouă în Indexul de Embedding numită `similarity`. Coloana `similarity` conține similaritatea cosinus între Embedding-ul interogării și Embedding-ul pentru fiecare segment video.
4. Apoi, Indexul de Embedding este filtrat după coloana `similarity`. Indexul de Embedding este filtrat pentru a include doar videoclipurile care au o similaritate cosinus mai mare sau egală cu 0.75.
5. În final, Indexul de Embedding este sortat după coloana `similarity` și sunt returnate primele 5 videoclipuri.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Această funcție este foarte simplă, ea doar afișează rezultatele interogării de căutare.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Mai întâi, Indexul de Încapsulare este încărcat într-un Dataframe Pandas.  
2. Apoi, utilizatorului i se solicită să introducă o interogare.  
3. Apoi, funcția `get_videos` este apelată pentru a căuta în Indexul de Încapsulare interogarea.  
4. În final, funcția `display_results` este apelată pentru a afișa rezultatele utilizatorului.  
5. Utilizatorului i se solicită apoi să introducă o altă interogare. Acest proces continuă până când utilizatorul introduce `exit`.  

![](../../../../translated_images/ro/notebook-search.1e320b9c7fcbb0bc.webp)

Vi se va solicita să introduceți o interogare. Introduceți o interogare și apăsați enter. Aplicația va returna o listă de videoclipuri relevante pentru interogare. Aplicația va returna, de asemenea, un link către locul din videoclip unde se află răspunsul la întrebare.

Iată câteva interogări pe care le puteți încerca:

- Ce este Azure Machine Learning?  
- Cum funcționează rețelele neuronale convoluționale?  
- Ce este o rețea neuronală?  
- Pot folosi Jupyter Notebooks cu Azure Machine Learning?  
- Ce este ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim pentru acuratețe, vă rugăm să rețineți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa nativă trebuie considerat sursa autorizată. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm răspunderea pentru eventualele neînțelegeri sau interpretări greșite rezultate din utilizarea acestei traduceri.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
